# Programação Paralela e Distribuída

Hélio - DC/UFSCar - 2023

# Processamento vetorial em CPU com instruções SIMD

De acordo com a [taxonomia de Flyyn](https://en.wikipedia.org/wiki/Flynn%27s_taxonomy), SIMD (*Single Instruction, Multiple Data*) é um modo de execução que está associado à realização simultânea da mesma operação sobre diferentes dados (paralelismo de dados).  

Originalmente, este recurso surgiu em alguns supercomputadores vetoriais na década de 1970, e trata-se de uma forma de paralelismo com granularidade fina.

Com a evolução das tecnologias de construção de processadores e o potencial para otimização de operações aritméticas em estruturas vetoriais, comuns em muitas aplicações, CPUs de estações de trabalho passaram também a oferecer instruções para manipulação de dados no modelo SIMD.

Na linha dos processadores dos PCs, por exemplo, essa tecnologia surgiu no final da década 1990, com as extensões MMX, seguindo-se com a criação de outras versões desse recurso, com suporte para manipular mais dados e com mais bits ao mesmo tempo. Outros processadores também têm recursos equivalentes, chamados de instruções de **vetorização** (*vectorization*) [1].

Basicamente, essas extensões incluem registradores adicionais, mais longos, e uma lógica de circuito que permite realizar algumas operações aritméticas sobre várias partes desses registradores ao mesmo tempo.  Ou seja, operar no modo SIMD.

Há várias formas para se explorar os recursos de vetorização de um processador, começando por indicar ao compilador que, se possível, faça isso na geração de código.


[1] [https://en.wikipedia.org/wiki/Automatic_vectorization](https://en.wikipedia.org/wiki/Automatic_vectorization)



## Identificando extensões de suporte à vetorização

Como ilustrado em [2], para usar os recursos de vetorização, é preciso que tanto o processador quanto o compilador disponíveis tenham suporte. Os comandos a seguir ilustram esta verificação.

[2] [https://tech.io/playgrounds/283/sse-avx-vectorization/prerequisites](https://tech.io/playgrounds/283/sse-avx-vectorization/prerequisites)

```
In the CPU flag capabilities, we'll search for the avx flag. This identifies the CPU as AVX compatible.

If you have avx2 that means the CPU allows AVX2 extensions. AVX is enough to have 8x32bit float vectors.

AVX2 adds 256bits vectors for integers (8x32bit integers for example).
Nevertheless, 256bit integer vectors seem to be executed the same as two 128bit vectors,
so performance is not greatly improved from SSE 128bit integer vectors.

In the GCC capabilities we'll search for the #define __AVX__ 1 pragma.
This indicates that the AVX branches will be enabled.

Alway use -march=native or -mavx !! If you run GCC without the correct march you won't get the __AVX__ flag!!!
Default GCC parameters are generic, and without the flag it won't enable AVX even if the CPU is AVX capable.

Finally, we recheck that Linux Kernel is 2.6.30 or greater. A kernel 4.4.0 or greater is ideal.
```



In [ ]:
# CPU flag detection
! echo "** Getting CPU flag capabilities and number of cores"
! cat /proc/cpuinfo  | egrep "(flags|model name|vendor)" | sort | uniq -c
# Compiler capabilities. -march=native is required!
! echo; echo "** Getting GCC capabilities"
! gcc -march=native -dM -E - < /dev/null | egrep "SSE|AVX" | sort
# OS kernel version
! echo; echo "** Getting OS Kernel Version"
! uname -a

** Getting CPU flag capabilities and number of cores
      2 flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
      2 model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
      2 vendor_id	: GenuineIntel

** Getting GCC capabilities
#define __AVX__ 1
#define __AVX2__ 1
#define __MMX_WITH_SSE__ 1
#define __SSE__ 1
#define __SSE2__ 1
#define __SSE2_MATH__ 1
#define __SSE3__ 1
#define __SSE4_1__ 1
#define __SSE4_2__ 1
#define __SSE_MATH__ 1
#define __SSSE3__ 1

** Getting OS Kernel Version
Linux a17106b501e9 5.15.120+ #1 SMP Wed Aug 30 11:19:59 UTC 2023 x86_

# Auto-vetorização usando suporte do compilador

Os exemplos apresentados neste documento são baseados nos materiais a seguir, que ilustram aspectos da auto-vetorização realizada pelo compilador.

https://blog.qiqitori.com/2018/05/matrix-multiplication-using-gccs-auto-vectorization/

https://www.codingame.com/playgrounds/283/sse-avx-vectorization/autovectorization

Há também informações extraídas do manual do compilador gcc.

<br>

Para os testes, será usado um programa simples de multiplicação de matrizes, com cálculo percorrendo as colunas da matriz B, e uma versão que considera que a matriz B está **transposta**, de forma que os elementos originais das colunas podem ser obtidos sequencialmente, percorrendo as linhas da matriz B.

De maneira geral, o ojbetivo desses testes é avaliar a **capacidade do compilador** em utilizar as instruções vetoriais do processador na geração de código.

Nesse exemplo, não há dicas do programador via **#pragmas**, apenas a passagem de parâmetros na compilação solicitando a aplicação das técnicas de otimização que exploram, entre outras coisas, a vetorização.

In [ ]:
%%writefile mm.c

#include <stdio.h>

#define TAM 1024

float matrix_a[TAM][TAM];
float matrix_b[TAM][TAM];
float matrix_c[TAM][TAM];

int main(int argc, char **argv)
{
  int i,j,k;

  for (i = 0; i < TAM; i++)      // inicia matrizes
    for (j = 0; j < TAM; j++) {
      matrix_a[i][j] = 0.1f;
      matrix_b[i][j] = 0.2f;
      matrix_c[i][j] = 0.0f;       // pode ser substituído por memset (0...)
    }

  for (i=0; i < TAM; i++)      // percorre as linhas de A para calcular linhas de C
    for (j=0; j < TAM; j++)    // percorre as colunas de B para calcular colunas de C
      for (k=0; k < TAM; k++)  // percorre as colunas da linha de A e as linhas da coluna de B
        matrix_c[i][j] += matrix_a[i][k]*matrix_b[k][j];

  for (i=0; i < TAM; i++) {
    for (j=0; j < TAM; j++)
      printf("%f ", matrix_c[i][j]);
    printf("\n");
  }

  return 0;
 }

Writing mm.c


In [ ]:
# Compilação do programa
! gcc -Wall -o mm mm.c
! time ./mm > /dev/null


real	0m13.665s
user	0m13.526s
sys	0m0.014s


Será que há otimizações que o compilador possa fazer para gerar código mais eficiente, que resulte em menor tempo de conclusão?

No bloco a seguir, o código da multiplicação, mostrado acima (mm.c) será compilado usando apenas diferentes valores para o parâmetro de otimização do compilador.

O programa gerado em cada caso é executado 2 vezes via comando ***time***, que vai medir os tempos de execução.


In [ ]:
! for i in {0,1,2,3}; do \
    echo Compilando com -O$i && \
    if [ ! mm$i -nt mm.c ]; then gcc -O$i mm.c -o mm$i ; fi && \
    time -p ./mm$i > /dev/null && echo && \
    time -p ./mm$i > /dev/null && echo ; \
  done

Compilando com -O0
real 14.93
user 14.76
sys 0.01

real 14.13
user 14.00
sys 0.01

Compilando com -O1
real 4.54
user 4.50
sys 0.00

real 5.05
user 4.97
sys 0.00

Compilando com -O2
real 4.33
user 4.26
sys 0.01

real 4.37
user 4.33
sys 0.00

Compilando com -O3
real 1.41
user 1.39
sys 0.00

real 1.99
user 1.93
sys 0.01



Como se pode ver, as diferenças com a otimização são MUITO significativas em relação ao código não otimizado!

Mas quais otimizações será que foram aplicadas pelo compilador na geração do código?

# Usando otimizações específicas de vetorização

Nos blocos a seguir, um parâmetro a mais é usando na compilação do programa, para pedir ao compilador que indique quais otimizações foram aplicadas na geração do código. Com **gcc**, isso é feito com o parâmetro ***-fopt-info-optall-optimized***.

Na linha de comando, é passado o parâmetro ***-O3***, que explicita o nível de otimização 3, e o parâmetro ***-ftree-vectorize***, que indica ao compilador para tentar aplicar as otimizações de *loops*. O parâmetro ***-ftree-vectorizer-verbose=x*** indica o nível de detalhes exibidos sobre a aplicação desta técnica, e o parâmetro ***-fopt-info-optall-optimized*** indica as otimizações aplicadas.


Ah, para saber mais sobre as opções de otimização do gcc, veja: [https://gcc.gnu.org/onlinedocs/gcc/Optimize-Options.html](https://gcc.gnu.org/onlinedocs/gcc/Optimize-Options.html)

<br>

```
-ftree-vectorize : Perform vectorization on trees.
  This flag enables -ftree-loop-vectorize and -ftree-slp-vectorize if not explicitly specified.

-ftree-loop-vectorize : Perform loop vectorization on trees.
  This flag is enabled by default at -O2 and by -ftree-vectorize, -fprofile-use, and -fauto-profile.

-ftree-slp-vectorize : Perform basic block vectorization on trees.
  This flag is enabled by default at -O2 and by -ftree-vectorize, -fprofile-use, and -fauto-profile.
```


In [ ]:
# Experimentando otimizações de vetorização pelo compilador
# -ftree-vectorize is a simple alias to -ftree-loop-vectorize + -ftree-slp-vectorize
! gcc -g mm.c -o mm -O3 -fopt-info-optall-optimized -ftree-vectorize -ftree-vectorizer-verbose=5
! time ./mm > /dev/null

mm.c:29:5: optimized:   Inlining printf/15 into main/26 (always_inline).
mm.c:28:7: optimized:   Inlining printf/15 into main/26 (always_inline).
mm.c:14:17: optimized: Loop nest 1 distributed: split to 2 loops and 1 library calls.
mm.c:22:17: optimized: loop vectorized using 16 byte vectors
mm.c:15:19: optimized: loop vectorized using 16 byte vectors
mm.c:15:19: optimized: loop vectorized using 16 byte vectors

real	0m1.390s
user	0m1.362s
sys	0m0.013s


Experimentando mais com as opções de otimização, é possível verificar tanto quais otimizações foram aplicadas quanto aquelas que não foram, incluindo os motivos que impediram a paralelização.

O parâmetro ***-fopt-info-vec***, ou ***-fopt-info-vec-optimized***, exibe informações específicas sobre os *loops* que foram vetorizados.

In [ ]:
# -fopt-info-vec or -fopt-info-vec-optimized: The compiler will log which loops (by line N°) are being vector optimized.
# ! gcc mm.c -o mm -O3 -fopt-info-vec-optimized
! gcc mm.c -o mm -O3 -fopt-info-vec

mm.c:22:17: optimized: loop vectorized using 16 byte vectors
mm.c:15:19: optimized: loop vectorized using 16 byte vectors
mm.c:15:19: optimized: loop vectorized using 16 byte vectors


Já o parâmetro ***-fopt-info-vec-missed*** indica as oportunidades buscadas mas que não puderam ser exploradas, associadas aos motivos encontrados.


In [ ]:
# -fopt-info-vec-missed: Detailed info about loops not being vectorized, and a lot of other detailed information.
! gcc mm.c -o mm -O3 -fopt-info-vec-missed

mm.c:26:15: missed: couldn't vectorize loop
/usr/include/x86_64-linux-gnu/bits/stdio2.h:112:10: missed: statement clobbers memory: __printf_chk (1, "%f ", _7);
mm.c:27:17: missed: couldn't vectorize loop
/usr/include/x86_64-linux-gnu/bits/stdio2.h:112:10: missed: statement clobbers memory: __printf_chk (1, "%f ", _7);
mm.c:21:15: missed: couldn't vectorize loop
mm.c:21:15: missed: not vectorized: multiple nested loops.
mm.c:23:19: missed: couldn't vectorize loop
mm.c:23:19: missed: outer-loop already vectorized.
mm.c:14:17: missed: couldn't vectorize loop
mm.c:17:22: missed: not vectorized: complicated access pattern.
mm.c:14:17: missed: couldn't vectorize loop
mm.c:16:22: missed: not vectorized: complicated access pattern.
mm.c:18:22: missed: statement clobbers memory: __builtin_memset (&matrix_c, 0, 4194304);
/usr/include/x86_64-linux-gnu/bits/stdio2.h:112:10: missed: statement clobbers memory: __printf_chk (1, "%f ", _7);
/usr/include/x86_64-linux-gnu/bits/stdio2.h:112:10: missed: s

O parâmetro ***-fopt-info-vec-note***, por sua vez, mostra detalhes sobre os *loops* e as otimizações realizadas.





In [ ]:
# -fopt-info-vec-note: Detailed info about all loops and optimizations being done.
! gcc mm.c -o mm -O3 -fopt-info-vec-note

mm.c:10:5: note: vectorized 3 loops in function.
mm.c:26:15: note: ***** Analysis failed with vector mode V4SF
mm.c:26:15: note: ***** Skipping vector mode V16QI, which would repeat the analysis for V4SF


***-fopt-info-vec-all*** exibe todas informações mostradas pelos parâmetros anteriores.

Prepare-se!

In [ ]:
# -fopt-info-vec-all: All previous options together.
! gcc mm.c -o mm -O3 -fopt-info-vec-all

mm.c:26:15: missed: couldn't vectorize loop
/usr/include/x86_64-linux-gnu/bits/stdio2.h:112:10: missed: statement clobbers memory: __printf_chk (1, "%f ", _7);
mm.c:27:17: missed: couldn't vectorize loop
/usr/include/x86_64-linux-gnu/bits/stdio2.h:112:10: missed: statement clobbers memory: __printf_chk (1, "%f ", _7);
mm.c:21:15: missed: couldn't vectorize loop
mm.c:21:15: missed: not vectorized: multiple nested loops.
mm.c:22:17: optimized: loop vectorized using 16 byte vectors
mm.c:23:19: missed: couldn't vectorize loop
mm.c:23:19: missed: outer-loop already vectorized.
mm.c:14:17: missed: couldn't vectorize loop
mm.c:17:22: missed: not vectorized: complicated access pattern.
mm.c:15:19: optimized: loop vectorized using 16 byte vectors
mm.c:14:17: missed: couldn't vectorize loop
mm.c:16:22: missed: not vectorized: complicated access pattern.
mm.c:15:19: optimized: loop vectorized using 16 byte vectors
mm.c:10:5: note: vectorized 3 loops in function.
mm.c:18:22: missed: statement clob

## Otimizações de autovetorização GCC

[GCC Autovectorization flags](https://www.codingame.com/playgrounds/283/sse-avx-vectorization/autovectorization)

    GCC is an advanced compiler, and with the optimization flags -O3 or -ftree-vectorize
    the compiler will search for loop vectorizations (remember to specify the -mavx flag too).
    The source code remains the same, but the compiled code by GCC is completely different.

    GCC won't log anything about automatic vectorization unless some flags are enabled.
    If you need details of autovectorization results you can use the compiler flags:

    -fopt-info-vec or -fopt-info-vec-optimized: The compiler will log which loops (by line N°) are being vector optimized.
    -fopt-info-vec-missed: Detailed info about loops not being vectorized, and a lot of other detailed information.
    -fopt-info-vec-note: Detailed info about all loops and optimizations being done.
    -fopt-info-vec-all: All previous options together.

    NOTE: There are similar -fopt-info-[options]-optimized flags for other compiler optimizations,
    like inline: -fopt-info-inline-optimized


## Mais sobre otimizações do gcc

https://gcc.gnu.org/onlinedocs/gcc/Optimize-Options.html

# Trabalhando com a matriz transposta e acessos contíguos

Considerando o acesso não contíguo aos dados da matriz B, percorrida em suas colunas, seria de se esperar que o compilador não conseguisse fazer a vetorização do *loop* interno da multiplicação tradicional (for i... for j... for k...)

Examinando os logs do compilador, contudo, vê-se que houve alguma vetorização deste *loop*.

Para testar o efeito do acesso contíguo também aos dados da matriz B, uma nova versão do código da multiplicação é apresentada a seguir.

Considerando matrizes quadradas, a geração da versão transposta geraria a troca dos elementos B[i][j] por B[j][i]. Supondo que essa troca foi feita, ou que a matriz B já é uma versão transposta, a única mudança no código é o modo em que os elementos da matriz B são percorridos nas operações de multiplicação e soma para cálculo de cada elemento de C.

Tendo a matriz B na forma transposta, é possível ajustar o laço interno de cálculo dos elementos da matriz C para que os acessos aos elementos ocorram de forma contígua, como segue:

```
result[i][j] += matrix_a[i][k] * matrix_b[j][k];
```



    [https://blog.qiqitori.com/2018/05/matrix-multiplication-using-gccs-auto-vectorization/]

    As you can see, when we access matrix_a, we access matrix_a[i][0], then matrix_a[i][1], matrix_a[i][2], matrix_a[i][3], and so on until we have hit the end. This is nice and sequential memory access, and is much faster than haphazard (“random”) accesses.

    In matrix_b, we have somewhat haphazard accesses. The first access is matrix_b[0][j], the second access is (in our example) 1024 bytes away from the first, matrix_b[1][j], then another 1024 bytes away at matrix_b[2][j], etc. There is a 1024 byte gap between every access. This kind of access is slow. It ruins the CPU’s caching system.

    This is why matrix_b will often be transposed in matrix multiplication code. If you transpose the matrix, the rows will be the columns and the columns the rows, thus you get nice and sequential access to matrix_b. (In our demonstration code, we are using square matrices with the same values everywhere, so we don’t actually have to do any copying work, as matrix_b is the same transposed or not. So all we have to do is swap the indices.)



In [ ]:
%%writefile mmT.c

#include <stdio.h>

float matrix_a[1024][1024];
float matrix_b[1024][1024];
float result_matrix[1024][1024];

int main(int argc, char **argv)
{
  int i,j,k;

  for (i = 0; i < 1024; i++) {        // inicia matrizes
    for (j = 0; j < 1024; j++) {
      matrix_a[i][j] = 0.1f;
      matrix_b[i][j] = 0.2f;
      result_matrix[i][j] = 0.0f;     // pode ser substituído por memset (0...)
    }
  }

  for (i = 0; i < 1024; i++) {     // iterate over rows of matrix A/result matrix
    for (j = 0; j < 1024; j++) {   // iterate over columns matrix B/result matrix
      for (k = 0; k < 1024; k++) { // iterate over colums of matrix A and COLUMNS of matrix BT
        result_matrix[i][j] += matrix_a[i][k] * matrix_b[j][k];
      }
    }
  }

  for (i = 0; i < 1024; i++) {
    for (j = 0; j < 1024; j++) {
      printf("%f ", result_matrix[i][j]);
    }
    printf("\n");
  }

  return 0;
 }

Writing mmT.c


Pensemos na vetorização, supondo que o compilador vai vetorizar o *loop*, agora tendo tanto os dados a serem manipulados das linhas da matriz A e das colunas da matriz (transposta, agora nas linhas de B) armazenados de forma contígua na memória.

Para tanto, é possível usar parâmetros de otimização do compilador:


In [ ]:
# -ftree-vectorize is a simple alias to -ftree-loop-vectorize + -ftree-slp-vectorize
! gcc -O3 mmT.c -o mmT -fopt-info-optall-optimized -ftree-vectorize -ftree-vectorizer-verbose=5
! time ./mmT > /dev/null

mmT.c:32:5: optimized:   Inlining printf/15 into main/26 (always_inline).
mmT.c:30:7: optimized:   Inlining printf/15 into main/26 (always_inline).
mmT.c:12:17: optimized: Loop nest 1 distributed: split to 2 loops and 1 library calls.
mmT.c:22:21: optimized: loop vectorized using 16 byte vectors
mmT.c:13:19: optimized: loop vectorized using 16 byte vectors
mmT.c:13:19: optimized: loop vectorized using 16 byte vectors

real	0m2.206s
user	0m2.172s
sys	0m0.013s


Novamente, o parâmetro ***-fopt-info-vec-note*** mostra detalhes sobre os *loops* e as otimizações realizadas.

In [ ]:
# -fopt-info-vec-note: Detailed info about all loops and optimizations being done.
! gcc mmT.c -o mmT -O3 -fopt-info-vec-note

mmT.c:8:5: note: vectorized 3 loops in function.
mmT.c:28:17: note: ***** Analysis failed with vector mode V4SF
mmT.c:28:17: note: ***** Skipping vector mode V16QI, which would repeat the analysis for V4SF


O parâmetro ***-fopt-info-vec-missed*** indica as oportunidades buscadas mas que não puderam ser exploradas, associadas aos motivos encontrados.

In [ ]:
# -fopt-info-vec-missed: Detailed info about loops not being vectorized, and a lot of other detailed information.
! gcc mmT.c -o mmT -O3 -fopt-info-vec-missed

mmT.c:28:17: missed: couldn't vectorize loop
/usr/include/x86_64-linux-gnu/bits/stdio2.h:112:10: missed: statement clobbers memory: __printf_chk (1, "%f ", _7);
mmT.c:29:19: missed: couldn't vectorize loop
/usr/include/x86_64-linux-gnu/bits/stdio2.h:112:10: missed: statement clobbers memory: __printf_chk (1, "%f ", _7);
mmT.c:20:17: missed: couldn't vectorize loop
mmT.c:20:17: missed: not vectorized: multiple nested loops.
mmT.c:21:19: missed: couldn't vectorize loop
mmT.c:23:60: missed: not vectorized: complicated access pattern.
mmT.c:12:17: missed: couldn't vectorize loop
mmT.c:15:22: missed: not vectorized: complicated access pattern.
mmT.c:12:17: missed: couldn't vectorize loop
mmT.c:14:22: missed: not vectorized: complicated access pattern.
mmT.c:16:27: missed: statement clobbers memory: __builtin_memset (&result_matrix, 0, 4194304);
/usr/include/x86_64-linux-gnu/bits/stdio2.h:112:10: missed: statement clobbers memory: __printf_chk (1, "%f ", _7);
/usr/include/x86_64-linux-gnu/bi

Aparentemente, o loop interno da multiplicação não foi vetorizado.

# Outras otimizações

Os 2 blocos de execução a seguir ilustram chamadas de compilação em que são feitas referências explícitas às extensões de vetorização SSE e AVX.

Uma diferença entre essas extensões é que AVX tem registradores de 256 bits, o que permite que mais operações sejam realizadas em paralelo, o dobro do que é oferecido pelos registradores de 128 bits de SSE.

In [ ]:
# -O3, SSE auto-vectorization, straight
! gcc -O3 -fopt-info-optall-optimized -ftree-vectorize -msse -o mm mm.c
! time ./mm > /dev/null

mm.c:29:5: optimized:   Inlining printf/15 into main/26 (always_inline).
mm.c:28:7: optimized:   Inlining printf/15 into main/26 (always_inline).
mm.c:14:17: optimized: Loop nest 1 distributed: split to 2 loops and 1 library calls.
mm.c:22:17: optimized: loop vectorized using 16 byte vectors
mm.c:15:19: optimized: loop vectorized using 16 byte vectors
mm.c:15:19: optimized: loop vectorized using 16 byte vectors

real	0m1.819s
user	0m1.767s
sys	0m0.011s


No bloco a seguir, solicita-se ao compilador que utilize explicitamente as extensões AVX.

In [ ]:
# -O3, AVX auto-vectorization, straight
! gcc -O3 -fopt-info-optall-optimized -ftree-vectorize -mavx -o mm mm.c
! time ./mm > /dev/null

mm.c:29:5: optimized:   Inlining printf/15 into main/26 (always_inline).
mm.c:28:7: optimized:   Inlining printf/15 into main/26 (always_inline).
mm.c:14:17: optimized: Loop nest 1 distributed: split to 2 loops and 1 library calls.
mm.c:22:17: optimized: loop vectorized using 32 byte vectors
mm.c:15:19: optimized: loop vectorized using 32 byte vectors
mm.c:15:19: optimized: loop vectorized using 32 byte vectors

real	0m1.084s
user	0m1.062s
sys	0m0.011s


# Usando vetorização e OpenMP

Para esse problema, é claro que também podemos usar OpenMP, por exemplo, para paralelizar o código usando diversas *threads*.

Para gerar mais carga, as multiplicações agora envolverão matrizes maiores (2048x2048).

In [ ]:
%%writefile mmomp.c

#include <stdio.h>

#define DIM 2048

float matrix_a[DIM][DIM];
float matrix_b[DIM][DIM];
float matrix_c[DIM][DIM];

int main(int argc, char **argv)
{
  int i,j,k;

  #pragma omp parallel for private(j)
  for (i = 0; i < DIM; i++) {           // inicia matrizes
    for (j = 0; j < DIM; j++) {
      matrix_a[i][j] = 0.1f;
      matrix_b[i][j] = 0.2f;
      matrix_c[i][j] = 0.0f;       // pode ser substituído por memset (0...)
    }
  }

  #pragma omp parallel for private(j,k)
  for (i = 0; i < DIM; i++)          // iterate over rows of matrix A/result matrix
    for (j = 0; j < DIM; j++)        // iterate over columns matrix B/result matrix
      for (k = 0; k < DIM; k++)      // iterate over columns of matrix A and rows of matrix B
        matrix_c[i][j] += matrix_a[i][k]*matrix_b[k][j];

  for (i = 0; i < DIM; i++) {
    for (j = 0; j < DIM; j++)
      printf("%f ", matrix_c[i][j]);
    printf("\n");
  }

  return 0;
 }

Writing mmomp.c


In [ ]:
! gcc -Wall -O3 mmomp.c -o mmomp -fopt-info-optall-optimized -ftree-vectorize
! time ./mmomp > /dev/null
! echo; echo "Agora com OpenMP:"; echo
! gcc -Wall -O3 mmomp.c -o mmomp -fopt-info-optall-optimized -ftree-vectorize -fopenmp
! echo; echo 2 threads:
! export OMP_NUM_THREADS=2 && time ./mmomp > /dev/null
! echo; echo 4 threads:
! export OMP_NUM_THREADS=4 && time ./mmomp > /dev/null

mmomp.c: In function ‘main’:
mmomp.c:14: warning: ignoring ‘#pragma omp parallel’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunknown-pragmas-Wunknown-pragmas]8;;]
   14 |   #pragma omp parallel for private(j)
      | 
mmomp.c:23: warning: ignoring ‘#pragma omp parallel’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunknown-pragmas-Wunknown-pragmas]8;;]
   23 |   #pragma omp parallel for private(j,k)
      | 
mmomp.c:32:5: optimized:   Inlining printf/15 into main/26 (always_inline).
mmomp.c:31:7: optimized:   Inlining printf/15 into main/26 (always_inline).
mmomp.c:15:17: optimized: Loop nest 1 distributed: split to 2 loops and 1 library calls.
mmomp.c:25:19: optimized: loop vectorized using 16 byte vectors
mmomp.c:16:19: optimized: loop vectorized using 16 byte vectors
mmomp.c:16:19: optimized: loop vectorized using 16 byte vectors

real	0m26.374s
user	0m25.911s
sys	0m0.045s

Agora com OpenMP:

mmomp.c:32:5: optimized:   Inlining 

## Operações SIMD com OpenMP

Uma outra construção específica de OpenMP é a diretiva [SIMD](https://www.openmp.org/spec-html/5.0/openmpsu42.html), que indica ao compilador que o loop a seguir pode ser executado usando instruções SIMD do processador.

```
#pragma omp simd [clause[ [,] clause] ... ] new-line
   for-loops
```

Esta diretiva pode ser declarada em qualquer trecho do código, acima de *for loops* que venham a ser encontrados pelas ***tasks***.


***Obs***: pelo que testei, contudo, o código gerado não explorou instruções SIMD do processador, salvo se os parâmetros -O2 ou -O3 fossem utilizados também.
Neste caso, o código SIMD gerado foi o mesmo que sem utilizar a diretiva ***omp simd*** :-(

# MM usando armazenamento contíguo

O exemplo a seguir procura explorar o armazenamento de matrizes como ponteiros. Para tanto, supõe que a matriz está armazenada como uma sequência de linhas.

Uma restrição deste tipo de armazenamento é que, ao tratar as operações com as matrizes (ponteiros), o compilador não tem certeza se os dados apontados por este ponteiro não são acessados por outras referências (*aliases*) . Deste modo, operações de vetorização, por exemplo, podem não ser feitas, por motivo de segurança.


Para tentar evitar que a vetorização deixe de ocorrer, uma opção é usar o qualificador \_\_restrict__.

```
The restrict keyword may be used to assert that the memory referenced by a pointer is
not aliased, i.e. that it is not accessed in any other way.
```


https://en.wikipedia.org/wiki/Restrict

```
In the C programming language, restrict is a keyword, introduced by the C99 standard,
that can be used in pointer declarations.
By adding this type qualifier, a programmer hints to the compiler that for the lifetime of the
pointer, no other pointer will be used to access the object to which it points.
This allows the compiler to make optimizations (for example, vectorization) that would not
otherwise have been possible.

```

<br>

https://gcc.gnu.org/onlinedocs/gcc/Loop-Specific-Pragmas.html

Loop-Specific Pragmas

\#pragma GCC ivdep

    With this pragma, the programmer asserts that there are no loop-carried
    dependencies which would prevent consecutive iterations of the following
    loop from executing concurrently with SIMD (single instruction multiple
    data) instructions.

    For example, the compiler can only unconditionally vectorize the following loop with the pragma:

    void foo (int n, int *a, int *b, int *c)
    {
      int i, j;
    #pragma GCC ivdep
      for (i = 0; i < n; ++i)
        a[i] = b[i] + c[i];
    }

    In this example, using the restrict qualifier had the same effect.
    In the following example, that would not be possible. Assume k < -m or k >= m.
    Only with the pragma, the compiler knows that it can unconditionally vectorize the following loop:

    void ignore_vec_dep (int *a, int k, int c, int m)
    {
    #pragma GCC ivdep
      for (int i = 0; i < m; i++)
        a[i] = a[i + k] * c;
    }

\#pragma GCC unroll n

    You can use this pragma to control how many times a loop should be unrolled.
    It must be placed immediately before a for, while or do loop or a #pragma GCC ivdep,
    and applies only to the loop that follows. n is an integer constant expression
    specifying the unrolling factor. The values of 0 and 1 block any unrolling of the loop.



In [ ]:
%%writefile mm2.c

#include <stdio.h>
#include <stdlib.h>

#define TAM 1024

int
main(int argc, char **argv)
{
  int i,j,k;

//  float *__restrict__ matrix_a = malloc(TAM*TAM*sizeof(float));
//  float *__restrict__ matrix_b = malloc(TAM*TAM*sizeof(float));
//  float *__restrict__ matrix_c = malloc(TAM*TAM*sizeof(float));

  float * matrix_a = malloc(TAM*TAM*sizeof(float));
  float * matrix_b = malloc(TAM*TAM*sizeof(float));
  float * matrix_c = malloc(TAM*TAM*sizeof(float));

  for (i = 0; i < TAM * TAM; i++) {
    *(matrix_a +i) = 0.1f;
    *(matrix_b +i) = 0.2f;
    *(matrix_c +i) = 0.0f;
  }

  #pragma GCC ivdep
  for (i = 0; i < 1024; i++)
    for (j = 0; j < 1024; j++)
      for (k = 0; k < 1024; k++)
        matrix_c[i*1024+j] += matrix_a[i*1024 +k] * matrix_b[k*1024 +j];

  for (int i = 0; i < TAM; i++) {
    for (int j = 0; j < TAM; j++)
      printf("%f ", matrix_c[i*TAM+j]);
    printf("\n");
  }

  return 0;
}


Writing mm2.c


In [ ]:
# ! gcc --version
#
! gcc -Wall -g -O3 mm2.c -o mm2 -ftree-vectorize -fopt-info-optall-optimized
#
! echo
# -fopt-info-vec-missed: Detailed info about loops not being vectorized, and a lot of other detailed information.
! gcc mm2.c -o mm2 -O3 -fopt-info-vec-missed
#
# -fopt-info-vec-note: Detailed info about all loops and optimizations being done.
# ! gcc mm2.c -o mm2 -O3 -fopt-info-vec-note
# ! gcc mm2.c -o mm2 -O3
# ! time ./mm2 > /dev/null

mm2.c:35:5: optimized:   Inlining printf/15 into main/39 (always_inline).
mm2.c:34:7: optimized:   Inlining printf/15 into main/39 (always_inline).
mm2.c:20:17: optimized: Loop 1 distributed: split to 2 loops and 1 library calls.
mm2.c:28:19: optimized: loop vectorized using 16 byte vectors
mm2.c:20:17: optimized: loop vectorized using 16 byte vectors
mm2.c:20:17: optimized: loop vectorized using 16 byte vectors

mm2.c:32:21: missed: couldn't vectorize loop
/usr/include/x86_64-linux-gnu/bits/stdio2.h:112:10: missed: statement clobbers memory: __printf_chk (1, "%f ", _31);
mm2.c:33:23: missed: couldn't vectorize loop
/usr/include/x86_64-linux-gnu/bits/stdio2.h:112:10: missed: statement clobbers memory: __printf_chk (1, "%f ", _31);
mm2.c:27:3: missed: couldn't vectorize loop
mm2.c:27:3: missed: not vectorized: multiple nested loops.
mm2.c:29:21: missed: couldn't vectorize loop
mm2.c:29:21: missed: outer-loop already vectorized.
mm2.c:16:22: missed: statement clobbers memory: matrix_a_46

In [ ]:
# ! if $( ! apt list gdb | grep "installed" &> /dev/null ) ; then apt install -y gdb ; fi
! if [ ! mm2 -nt mm2.c ]; then gcc -Wall mm2.c -o mm2 -g -O3 -fopt-info-optall-optimized -ftree-vectorize ; fi
# ! gdb ./mm2 -ex "break 28" -ex "r"
! time ./mm2 > /dev/null


real	0m1.520s
user	0m1.493s
sys	0m0.015s


# Teste de linearização das matrizes


In [ ]:
%%writefile mm-sl.c

#include <stdio.h>
#include <stdlib.h>

#define TAM 1024
// #define TAM 2048

int
main(int argc, char **argv)
{
  int i,j,k;

  float *matrix_a = malloc(TAM*TAM*sizeof(float));
  float *matrix_b = malloc(TAM*TAM*sizeof(float));
  float *result_matrix = malloc(TAM*TAM*sizeof(float));

  for (i = 0; i < TAM * TAM; i++) {
    *(matrix_a +i) = 0.1f;
    *(matrix_b +i) = 0.2f;
    // *(result_matrix +i) = 0.0f;
  }

  for (i = 0; i < TAM; i++) {     // iterate over rows of matrix A/result matrix
    for (j = 0; j < TAM; j++) {   // iterate over columns matrix B/result matrix
      result_matrix[i*TAM+j] = 0.0;
      for (k = 0; k < TAM; k++) { // iterate over columns of matrix A and rows of matrix B
        result_matrix[i*TAM+j] += matrix_a[i*TAM +k] * matrix_b[j*TAM +k];
      }
    }
  }

  for (int i = 0; i < TAM; i++) {
    for (int j = 0; j < TAM; j++)
      printf("%f ", result_matrix[i*TAM+j]);
    printf("\n");
  }

  return 0;
}


Writing mm-sl.c


In [ ]:
%%writefile mm-loc.c

#include <stdio.h>
#include <stdlib.h>

#define TAM 1024
// #define TAM 2048

int
main(int argc, char **argv)
{
  int i,j,k;
  float aux;

  float *matrix_a = malloc(TAM*TAM*sizeof(float));
  float *matrix_b = malloc(TAM*TAM*sizeof(float));
  float *result_matrix = malloc(TAM*TAM*sizeof(float));

  for (i = 0; i < TAM * TAM; i++) {
    *(matrix_a +i) = 0.1f;
    *(matrix_b +i) = 0.2f;
    // *(result_matrix +i) = 0.0f;
  }

  for (i = 0; i < TAM; i++) { // iterate over rows of matrix A/result matrix
    for (j = 0; j < TAM; j++) { // iterate over columns matrix B/result matrix
      aux = 0.0;
      for (k = 0; k < TAM; k++) { // iterate over columns of matrix A and rows of matrix B
        // result_matrix[i*TAM+j] += matrix_a[i*TAM +k] * matrix_b[j*TAM +k];
        aux += matrix_a[i*TAM +k] * matrix_b[j*TAM +k];
      }
      result_matrix[i*TAM+j] = aux;
    }
  }

  for (int i = 0; i < TAM; i++) {
    for (int j = 0; j < TAM; j++)
      printf("%f ", result_matrix[i*TAM+j]);
    printf("\n");
  }

  return 0;
}


Writing mm-loc.c


In [ ]:
! if [ ! mm-sl -nt mm-sl.c ]; then gcc -Wall mm-sl.c -o mm-sl -O3; fi
! if [ ! mm-loc -nt mm-loc.c ]; then gcc -Wall mm-loc.c -o mm-loc -O3; fi
! time ./mm-sl > /dev/null
! time ./mm-loc > /dev/null


real	0m2.185s
user	0m2.156s
sys	0m0.012s

real	0m2.727s
user	0m2.636s
sys	0m0.015s


Como se vê, o desempenho com o uso de uma variável local, neste caso, com -O2 ou -O3, deixa de usar a possibilidade de somas parciais também vetorizadas no salvamento dos valores parciais nas linhas da matriz C.

Sem as otimizações de vetorização, o uso de variável parcial vale a pena, explorando a localidade e o uso do cache.

# mm ijk -> ikj

Por fim, já que estamos investigando otimizações na multiplicação de matrizes, que tal analisar também a ordem dos *loops* da multiplicação?

<br>

Será que a inversão dos *loops* intermediário e central tem algum efeito no tempo de execução da multiplicação de matrizes?

O que muda nos cálculos?

Com esta inversão, percorre-se cada elemento da linha de A, multiplicando-o pelo primeiro elemento das linhas de B, e acumulando este resultado parcial em cada elemento das colunas desta linha de C.

Para entender melhor os cálculos com valores parciais, veja: https://commons.wikimedia.org/wiki/Category:Animations_of_optimization_(image_set_by_Maxiantor).

<br>

Em suma, há um reúso máximo do valor de cada elemento das linhas de A, aproveitando também o cache na busca dos elementos da linha de B.

A questão é apenas se as somas parciais podem ser feitas em paralelo...

Será que ***#pragma omp [atomic](https://www.openmp.org/spec-html/5.1/openmpsu105.html#x138-1480002.19.7) update*** resolve?

In [ ]:
%%writefile ikj.c

#include <stdio.h>

#define TAM 1024

float matrix_a[TAM][TAM];
float matrix_b[TAM][TAM];
float matrix_c[TAM][TAM];

int main(int argc, char **argv)
{
  int i,j,k;

  for (i = 0; i < TAM; i++) {        // inicia matrizes
    for (j = 0; j < TAM; j++) {
      matrix_a[i][j] = 0.1f;
      matrix_b[i][j] = 0.2f;
      matrix_c[i][j] = 0.0f;     // pode ser substituído por memset (0...)
    }
  }

  for (i = 0; i < TAM; i++)      // percorre as linhas de C
      for (k = 0; k < TAM; k++)  // percorre colunas da linha de A e linhas da coluna de B
    for (j = 0; j < TAM; j++)    // percorre colunas de C
        matrix_c[i][j] += matrix_a[i][k] * matrix_b[k][j];


  for (i = 0; i < TAM; i++) {
    for (j = 0; j < TAM; j++) {
      printf("%f ", matrix_c[i][j]);
    }
    printf("\n");
  }

  return 0;
 }

Overwriting ikj.c


In [ ]:
! gcc -Wall ikj.c -o ikj -O3 -ftree-vectorize -fopt-info-optall-optimized # -fopt-info-optall-missed
! time ./ikj > /dev/null
! echo
! gcc -Wall ikj.c -o ikj-avx -O3 -mavx -fopt-info-optall-optimized
! time ./ikj-avx > /dev/null

ikj.c:32:5: optimized:   Inlining printf/15 into main/26 (always_inline).
ikj.c:30:7: optimized:   Inlining printf/15 into main/26 (always_inline).
ikj.c:23:21: optimized: applying unroll and jam with factor 2
ikj.c:14:17: optimized: Loop nest 1 distributed: split to 2 loops and 1 library calls.
ikj.c:24:19: optimized: loop vectorized using 16 byte vectors
ikj.c:24:19: optimized: loop vectorized using 16 byte vectors
ikj.c:15:19: optimized: loop vectorized using 16 byte vectors
ikj.c:15:19: optimized: loop vectorized using 16 byte vectors

real	0m1.388s
user	0m1.327s
sys	0m0.012s

ikj.c:32:5: optimized:   Inlining printf/15 into main/26 (always_inline).
ikj.c:30:7: optimized:   Inlining printf/15 into main/26 (always_inline).
ikj.c:23:21: optimized: applying unroll and jam with factor 2
ikj.c:14:17: optimized: Loop nest 1 distributed: split to 2 loops and 1 library calls.
ikj.c:24:19: optimized: loop vectorized using 32 byte vectors
ikj.c:24:19: optimized: loop vectorized using 32 byte